<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/sparse_jac_combineRows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numba import njit
import numpy as np
import jax
import jax.numpy as jnp
from itertools import permutations
import scipy as sp
from copy import deepcopy
from itertools import chain
np.set_printoptions(precision=2, linewidth=300, threshold=1e6, suppress= True)

In [34]:
def create_block_sparse(N_blocks_diag, block_size_lo, block_size_hi, Noff_diag):

    block_sizes = np.random.randint(low=block_size_lo, high=block_size_hi+1, size=N_blocks_diag)
    cum_pos = np.cumsum(block_sizes)
    cum_pos = np.r_[0, cum_pos]
    N = cum_pos[-1]

    diag_data = [np.random.uniform(size=(block_sizes[i], block_sizes[i])) for i in range(N_blocks_diag)]
    off_diag_blocks =np.asarray(tuple(set(tuple(permutations(tuple(range(N_blocks_diag)),2))) - set(tuple([i,i]) for i in range(N_blocks_diag))))
    np.random.shuffle(off_diag_blocks)
    off_diag_blocks=off_diag_blocks[:Noff_diag]
    off_data=[np.random.uniform(size=(block_sizes[i], block_sizes[j])) for i,j in off_diag_blocks]

    data = []
    col = []
    row=[]

    for i in range(N_blocks_diag):
        coo = sp.sparse.coo_array(diag_data[i])
        data.extend(coo.data)
        col.extend(coo.row+cum_pos[i])
        row.extend(coo.col+cum_pos[i])

    for i in range(Noff_diag):
        coo = sp.sparse.coo_array(off_data[i])
        data.extend(coo.data)
        brow, bcol = off_diag_blocks[i]
        col.extend(coo.row+cum_pos[brow])
        row.extend(coo.col+cum_pos[bcol])

    return sp.sparse.coo_array((data, (row, col)),shape=(N,N))


In [35]:
a=create_block_sparse(10, 2, 3, 10)

In [36]:
def get_combos(mat):
    mat=sp.sparse.csr_array(mat)
    nz = (mat!=0).astype(np.int8)
    c = (nz @ nz.T).astype(bool)
    N=mat.shape[0]
    cand = set(range(N))
    cols=[]
    rows=[]
    vecs=[]
    ls=[]
    while cand:
        vec = np.zeros(N)
        k = cand.pop()
        cand2 = deepcopy(cand)
        v=set(c.indices[c.indptr[k]:c.indptr[k+1]])
        matcols = mat.indices[mat.indptr[k]:mat.indptr[k+1]]
        col1=[matcols]
        row1=[np.full_like(matcols,k)]
        ls1=[k]
        while True:
            cand2-=v
            if not(cand2):
                break
            first=cand2.pop()
            v = set(c.indices[c.indptr[first]:c.indptr[first+1]])
            ls1.append(first)
            matcols=mat.indices[mat.indptr[first]:mat.indptr[first+1]]
            col1.append(matcols)
            row1.append(np.full_like(matcols,first))
            cand.remove(first)
        vec[ls1]=1.
        vecs.append(vec)
        cols.append(np.concatenate(col1))
        rows.append(np.concatenate(row1))
        ls.append(ls1)
    return ls, vecs, rows, cols

In [37]:
ls, vecs,rows,cols= get_combos(a)

In [38]:
a.todense()[ls[0],:]

array([[0.88, 0.04, 0.82, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.67, 0.11, 0.  , 0.  , 0.  , 0.5 , 0.06, 0.36, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.92, 0.22, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.38, 0.64, 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [ ]:

shape = (100, 100)
@jax.jit
def fn(x_flat):
    x = jnp.reshape(x_flat, shape)
    result = jax.scipy.signal.convolve2d(x, jnp.ones((3, 3)), mode="valid")
    return result.flatten() ** 2

x_flat = np.random.uniform(size=(shape[0] * shape[1]))
expected = jax.jit(jax.jacrev(fn))(x_flat)


In [43]:
ls, vecs,rows,cols= get_combos(expected)

In [44]:
_, vjp = jax.vjp(fn, x_flat)

In [45]:
data=[]
for vec,col in zip(vecs,cols):
    data.append(vjp(vec)[0][col])

data = np.concatenate(data)

In [46]:
data_rows = np.concatenate(rows)
data_cols=np.concatenate(cols)
jac = sp.sparse.coo_array((data,(data_rows, data_cols)))

In [47]:
np.allclose(jac.todense(),expected)

True

In [22]:
len(ls)

12